In [2]:
años = list(range(2013, 2024))

In [4]:
base_url = "https://hoopshype.com/salaries/players/"

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [1]:
# Temporadas desde 2013-2014 hasta 2022-2023 inclusive
temporadas = ["2013-2014", "2014-2015", "2015-2016", "2016-2017", "2017-2018","2018-2019", "2019-2020", "2020-2021", "2021-2022", "2022-2023"]

In [4]:
# Temporada de prueba
temporada = "2013-2014"
url = f"https://hoopshype.com/salaries/players/{temporada}/"

# Hacemos la solicitud HTTP
response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})

# Parseamos el contenido HTML
soup = BeautifulSoup(response.text, "html.parser")

# Mostramos una vista previa de la tabla (sólo para inspección)
tabla = soup.find("table")
print(tabla.prettify()[:1000])  # Mostramos los primeros 1000 caracteres


<table class="hh-salaries-ranking-table hh-salaries-table-sortable responsive">
 <thead>
  <tr class="table-index">
   <td class="rank">
   </td>
   <td class="name">
    Player
   </td>
   <td class="">
    2013/14
   </td>
   <td class="">
    2013/14(*)
   </td>
  </tr>
 </thead>
 <tbody>
  <tr>
   <td class="rank">
    1.
   </td>
   <td class="name">
    <a href="https://hoopshype.com/player/kobe-bryant/salary/">
     Kobe Bryant
    </a>
   </td>
   <td class="" data-value="30453000" style="color:black">
    $30,453,000
   </td>
   <td class="" data-value="40973907" style="color:black">
    $40,973,907
   </td>
  </tr>
  <tr>
   <td class="rank">
    2.
   </td>
   <td class="name">
    <a href="https://hoopshype.com/player/dirk-nowitzki/salary/">
     Dirk Nowitzki
    </a>
   </td>
   <td class="" data-value="22721381" style="color:black">
    $22,721,381
   </td>
   <td class="" data-value="30571167" style="color:black">
    $30,571,167
   </td>
  </tr>
  <tr>
   <td class="ra

In [5]:
# Buscamos todas las filas de la tabla (menos el encabezado)
filas = tabla.find("tbody").find_all("tr")

# Guardamos los resultados
salarios_temp = []

for fila in filas:
    columnas = fila.find_all("td")
    if len(columnas) >= 3:
        nombre = columnas[1].text.strip()
        salario = columnas[2].text.strip().replace("$", "").replace(",", "")
        if salario.isdigit():
            salario = int(salario)
        else:
            salario = None
        salarios_temp.append({"temporada": temporada, "jugador": nombre, "salario": salario})

# Mostramos los primeros resultados
pd.DataFrame(salarios_temp).head()


,temporada,jugador,salario
0,2013-2014,Kobe Bryant,30453000
1,2013-2014,Dirk Nowitzki,22721381
2,2013-2014,Gilbert Arenas,22346536
3,2013-2014,Amare Stoudemire,21679893
4,2013-2014,Carmelo Anthony,21490000


In [6]:
# Lista de temporadas a scrapear
temporadas = [
    "2013-2014", "2014-2015", "2015-2016", "2016-2017", "2017-2018",
    "2018-2019", "2019-2020", "2020-2021", "2021-2022", "2022-2023"
]

# Lista para guardar todos los salarios de todas las temporadas
todos_los_salarios = []

# Loop por cada temporada
for temporada in temporadas:
    print(f"Scrapeando temporada {temporada}...")
    url = f"https://hoopshype.com/salaries/players/{temporada}/"
    response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
    soup = BeautifulSoup(response.text, "html.parser")
    tabla = soup.find("table")
    
    try:
        filas = tabla.find("tbody").find_all("tr")
    except:
        print(f"No se pudo leer la tabla de la temporada {temporada}")
        continue

    for fila in filas:
        columnas = fila.find_all("td")
        if len(columnas) >= 3:
            nombre = columnas[1].text.strip()
            salario = columnas[2].text.strip().replace("$", "").replace(",", "")
            if salario.isdigit():
                salario = int(salario)
            else:
                salario = None
            todos_los_salarios.append({
                "temporada": temporada,
                "jugador": nombre,
                "salario": salario
            })
    
    time.sleep(1)  # descanso para no sobrecargar el servidor

# Convertimos todo a DataFrame
df_salarios = pd.DataFrame(todos_los_salarios)

# Mostramos los primeros resultados
df_salarios.head()


Scrapeando temporada 2013-2014...
Scrapeando temporada 2014-2015...
Scrapeando temporada 2015-2016...
Scrapeando temporada 2016-2017...
Scrapeando temporada 2017-2018...
Scrapeando temporada 2018-2019...
Scrapeando temporada 2019-2020...
Scrapeando temporada 2020-2021...
Scrapeando temporada 2021-2022...
Scrapeando temporada 2022-2023...


,temporada,jugador,salario
0,2013-2014,Kobe Bryant,30453000
1,2013-2014,Dirk Nowitzki,22721381
2,2013-2014,Gilbert Arenas,22346536
3,2013-2014,Amare Stoudemire,21679893
4,2013-2014,Carmelo Anthony,21490000


In [8]:
df_salarios 

,temporada,jugador,salario
0,2013-2014,Kobe Bryant,30453000
1,2013-2014,Dirk Nowitzki,22721381
2,2013-2014,Gilbert Arenas,22346536
3,2013-2014,Amare Stoudemire,21679893
4,2013-2014,Carmelo Anthony,21490000
...,...,...,...
5527,2022-2023,Gabe York,32171
5528,2022-2023,Ibou Badji,18226
5529,2022-2023,Tristan Thompson,16700
5530,2022-2023,RaiQuan Gray,5849


In [9]:
df_salarios.to_csv("salarios_nba_2013_2023.csv", index=False)
